# Exercise 1
Add Backpropagation to your MLP and train the model on the ZIP-Dataset.

In [19]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from statistics import mean
import random
from numpy import linalg as LA
from sklearn import decomposition
%matplotlib inline

**1. Initialization**
- define threshold for activation function (UNUSED), number of layers, number of neurons per layer
- for each layer, initialize a weight matrix with random numbers (dim = #neurons in previous leyer x #neurons in current layer) and a bias vector with ones

**2. Training**
- ***Feedforward***
    - Feed batch data through layers (f_i(w_i*x+b), using weight matrices w and activation functions f)
    - for every neuron/layer, store output value y_i and derivative y*_i
    
- ***Backpropagation*** 
    - Quantify network error (MSE = 1/2 * (y_n-t)^2)
    - Update weights w_i in layer i by product of backpropagated error, derivative, input vector and learning rate (SGD)
**3. Prediction/Inference**


In [313]:
class MLP:
    def __init__(self, depth, layer_width, threshold, learning_rate):
        """
        This constructor sets random network weights and checks if the input depth matches the provided layers.
        """
        self.threshold = threshold
        self.learning_rate = learning_rate
        self.depth = depth
        if not len(layer_width) == (depth + 1):
            raise Exception("'layer_width' needs to be of length 'depth' + 1")  
        self.layer_width = layer_width
        self.network_weights = []
        self.network_biases = []
        self.network_derivatives = [np.zeros((1,self.layer_width[0]))]
        self.network_outputs = [np.zeros((1,self.layer_width[0]))]
        width_prev = self.layer_width[0]
        for width in self.layer_width[1:]:
            self.network_weights.append(np.random.randn(width_prev, width)* np.sqrt(1. / width_prev))
            self.network_biases.append(np.zeros((1, width)))
            self.network_derivatives.append(np.zeros((1, width))) # tmp
            self.network_outputs.append(np.zeros((1, width)))
            width_prev = width
        
    def mean_squared_error(self, Y_m, T_m):
        """Quantify rrror after feedforward step."""
        return 1/2 * np.power((Y_m - T_m),2)
    
    def heaviside(self, X):
        """This Function is a tiny implementation of the heaviside step function."""
        return (X >= self.threshold).astype(int)
    
    def sigmoid(self, X):
        return 1/1+np.exp(X)
            
    def backpropagate(self, Error):
        """Backpropagate error and update weight matrices."""
        # from first to last layer
        for i in range(self.depth):
            print('Update weights in layer',i)
            print('derivatives * error:')
            print(self.network_derivatives[self.depth].T.shape, Error.shape)
            tmp = self.network_derivatives[self.depth].T*Error
            print(tmp.shape)
            print('get partial derivatives')
            for j in range(self.depth-1, i-1, -1):
                print('derivatives, weights, tmp')
                print(self.network_derivatives[j].shape,self.network_weights[j].shape,tmp.T.shape)
                tmp = self.network_derivatives[j]@self.network_weights[j]*tmp.T
                print(tmp.shape)
            print('update weights')
            # update weights: learning_rate*derivatives*weights*...*input
            print('lr, tmp (derivatives*weights), input')
            print(self.learning_rate, tmp.shape, self.network_outputs[i-1].shape)
            self.network_weights[i] = self.learning_rate * tmp * self.network_outputs[i-1] 
            # TODO: also update biases?
        return
    
    def feed_forward(self, X):
        """This Function passes the input X through all weights and returns the prediction vector."""
        X_i = X.copy()
        self.network_outputs[0] = X_i
        self.network_derivatives[0] = X_i
        for i in range(self.depth):
            # Compute weighted sum
            z_i = X_i @ self.network_weights[i] + self.network_biases[i]
            # Apply activation function
            #X_i = self.heaviside(z_i)
            X_i = self.sigmoid(z_i)
            # Store derivatives
            D_i = X_i*(1-X_i)
            self.network_outputs[i+1] = X_i
            self.network_derivatives[i+1] = D_i
        return X_i
    
    def train(self, X, Y, M):
        """Train the MLP on (X,Y) in M equally sized batches using feedforward and backpropagation with Stochastic Gradient Descent."""
        # Shuffle data indices and split in subsets of size M
        batch_indices = np.arange(X.shape[0])
        np.random.shuffle(batch_indices)
        batch_splits = np.array_split(batch_indices, M)
        for m in batch_splits:
            # fetch batch
            X_m = X[m]
            T_m = Y[m]
            Y_m = self.feed_forward(X_m) 
            # Quantify error
            E = self.mean_squared_error(Y_m.ravel(), T_m)
            # Backpropagate
            self.backpropagate(E)
        #raise Exception("The train function will be implemented in Assignment 9!!!")
        return 
    
    def predict(self, X):
        """This function passes the input X to the iteration function."""
        X_i = X.copy()
        for i in range(self.depth):
            # Compute weighted sum
            z_i = X_i @ self.network_weights[i] + self.network_biases[i]
            # Apply activation function
            #X_i = self.heaviside(z_i)
            X_i = self.sigmoid(z_i)
        return X_i.ravel()
    
    def accuracy(self, labels, predictions):
        """This function calculates the binary class accuracy for given true/predicted labels."""
        return np.mean(labels == predictions)

**Load ZIP data set**

In [260]:
path_to_train = '/Users/Eva/Downloads/zip.train'
path_to_test = '/Users/Eva/Downloads/zip.test'
training_data = np.array(pd.read_csv(path_to_train, sep=' ', header=None))
test_data = np.array(pd.read_csv(path_to_test, sep =' ',header=None))

X_train_zip, y_train_zip = training_data[:,1:-1], training_data[:,0]
X_test_zip, y_test_zip = test_data[:,1:], test_data[:,0]

# We only want to classify two different digits. You can choose which digits you want to classify youself

X_train_zip = X_train_zip[np.logical_or(y_train_zip == 0, y_train_zip == 1)]
y_train_zip = y_train_zip[np.logical_or(y_train_zip == 0, y_train_zip == 1)]

X_test_zip = X_test_zip[np.logical_or(y_test_zip == 0, y_test_zip == 1)]
y_test_zip = y_test_zip[np.logical_or(y_test_zip == 0, y_test_zip == 1)]

#### Classify the Zip-Dataset with the random initial weights

In [314]:
mlp_network = MLP(threshold=0.01, learning_rate=0.1, depth=2, layer_width=[X_train_zip.shape[1], 10, 1])

In [315]:
for i in range(mlp_network.depth):
    print('Layer:', i)
    print(mlp_network.network_weights[i].shape)
    print(mlp_network.network_biases[i].shape)
    print(mlp_network.network_outputs[i].shape)
    print(mlp_network.network_derivatives[i].shape)

Layer: 0
(256, 10)
(1, 10)
(1, 256)
(1, 256)
Layer: 1
(10, 1)
(1, 1)
(1, 10)
(1, 10)


In [316]:
(np.ones((10,10))@np.ones((10,1))).shape

(10, 1)

In [317]:
mlp_network.train(X_train_zip[:100,:], y_train_zip[:100], 10)

Update weights in layer 0
derivatives * error:
(1, 10) (10,)
(1, 10)
get partial derivatives
derivatives, weights, tmp
(10, 10) (10, 1) (10, 1)
(10, 1)
derivatives, weights, tmp
(10, 256) (256, 10) (1, 10)
(10, 10)
update weights
lr, tmp (derivatives*weights), input
0.1 (10, 10) (10, 1)
Update weights in layer 1
derivatives * error:
(1, 10) (10,)
(1, 10)
get partial derivatives
derivatives, weights, tmp
(10, 10) (10, 1) (10, 1)
(10, 1)
update weights
lr, tmp (derivatives*weights), input
0.1 (10, 1) (10, 256)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 10 is different from 256)

In [216]:
y_pred_mlp = mlp_network.predict(X_train_zip)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 10 is different from 256)

In [7]:
mlp_network.accuracy(y_train_zip, y_pred_mlp)

0.34015461573442474

In [8]:
np.unique(y_pred_mlp, return_counts=True)

(array([0, 1]), array([ 637, 1562]))

In [9]:
np.unique(y_train_zip, return_counts=True)

(array([0., 1.]), array([1194, 1005]))

#### Get a mean accuracy over multiple runs

In [27]:
acc_list_mlp = []
n_runs = 100
for i in range(n_runs):
    mlp_network = MLP(threshold=0.01, depth=2, layer_width=[X_train_zip.shape[1], 10, 1])
    y_pred_loop = mlp_network.predict(X_train_zip)
    acc_list_mlp.append(mlp_network.accuracy(y_train_zip, y_pred_loop))
print("Mean Acc over", n_runs, "runs, with random weights is:", np.mean(acc_list_mlp))

Mean Acc over 100 runs, with random weights is: 0.49954524783992726


### (a) Optimize width (the number of neurons in a hidden layer; it is usually the same for all of them) and depth of the network. Try to find a setting that trains in a reasonable time. Plot the loss.

### (b) Show some digits that are classified incorrectly.

### (c) Plot your first weight layer as a grayscale image.